In this homework, you'll explore using BERT and SentenceTransformers into the Lesk algorithm for word sense disambiguation.  (You'll likely want to run this on Colab.)

In [1]:
# Note: Christian and I worked on this code together

!pip install transformers
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for torch>=1.6.0 from https://files.pythonhosted.org/packages/16/dd/1bf10180ba812afa1aa7427466083d731bc37b9a1157ec929d0cfeef87eb/torch-2.1.0-cp311-none-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/23/84/46481327771d4f63feb59dd0d9e1cd6a42e985dbd371965f486a5bf9f323/torchvision-0.16.0-cp311-cp311-macosx_10_13_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 38.1 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.whl.metadata


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.6 MB/s eta 0:00:0000:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=3de07212045639b63db700e21ee54a3a4475e63dbf0902df123163c199451d8c
  Stored in directory: /Users/piadeshpande/Library/Caches/pip/wheels/ff/27/bf/ffba8b318b02d7f691a57084ee154e26ed24d012b0c7805881
Successfully built sentence-transformers


In [2]:
from transformers import BertModel, BertTokenizer
import numpy as np
from nltk.corpus import wordnet as wn
import nltk
from scipy.spatial.distance import cosine
import operator
import torch
from math import sqrt


In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/piadeshpande/nltk_data...


True

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

**Q1:** The Lesk algorithm we discussed in class uses information about the *context* around a term in calculating the similarity between a word in a sentence and a word in a dictionary gloss.  For instance, [Basile et al. 2014](https://www.aclweb.org/anthology/C/C14/C14-1151.pdf) use static word vectors to provide this context-level information, where we measure the similarity between a gloss g = $\{ g_1, \ldots, g_G \}$ and context c = $\{ c_1, \ldots, c_C \}$ as the cosine similarity between the sum of distributed representations:

$$
\cos \left(\sum_{i=1}^G g_i, \sum_{i=1}^C c_i  \right)
$$

However, over the past few weeks we've considered how contextual language models like BERT already provide a sentence-level contextualization for words.  So given a target sentence ("I withdrew money from the *bank*") with target term (bank), and a list of dictionary glosses/examples corresponding to different senses ("A bank is a financial institution" = bank1; "A bank is the side of a river" = bank2), let's adapt the Lesk algorithm to simply calculate the similarity between the average BERT embedding for all words in the target sentence (including the [CLS] and [SEP] tokens) and the average BERT embedding for all the words in the sense gloss (again including [CLS] and [SEP]):

$$
\cos \left({1 \over G}\sum_{i=1}^G BERT(g_{i}), {1 \over C} \sum_{j=1}^C BERT(c_{j}) \right)
$$


* The gloss for a synset can be found in `synset.definition()`.
* You can find the cosine similarity between two vectors below.
* `wn.synsets(word, pos=part_of_speech)` gets you a list of the synsets for a word with a specific part of speech (e.g., "n" for noun)
* Feel free to draw on the code you've already seen for getting the BERT embeddings for words (e.g., `3.embeddings/BERT.ipynb`).

In [5]:
def cosine_similarity(vec1, vec2):
  return np.dot(vec1, vec2)/(sqrt(np.dot(vec1, vec1)) * sqrt(np.dot(vec2, vec2)))

In [6]:
test = wn.synsets("test", pos="n")
test[1].definition()
#inputs = tokenizer(test[1].definition(), return_tensors="pt")
#tokens=tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
#term_idx=tokens.index(term)
#outputs = model(**inputs)
#res = outputs.last_hidden_state[0][term_idx].detach().numpy()
#res.shape()

'any standardized procedure for measuring sensitivity or memory or intelligence or aptitude or personality etc'

In [38]:
from nltk.corpus.reader.wordnet import Synset

def bert_lesk(word, sentence, part_of_speech):

# this function gets the BERT token representations for a given term within a larger string
    def get_bert_for_token(string, term):
        # tokenize
        inputs = tokenizer(string, return_tensors="pt")
        tokens=tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        # find the first location of the query term among those tokens (so we know which BERT rep to use)
        term_idx=tokens.index(term)
        outputs = model(**inputs)
        return outputs.last_hidden_state[0][term_idx].detach().numpy()
    context_vector=get_bert_for_token(sentence, word)
    synsets= wn.synsets(word,pos = part_of_speech)
    
    
    
# lastly, for every one of the synsets in the list, grab the BERT representation between the definition as the string and the input word
    # vals={}
   #  for synset in synsets:
      # code seems to be breaking here; where i get error related to term_idx; this should be vec2?
    #    vector=get_bert_for_token(synset.definition(), word)
    #    vals[synset]=cosine_similarity(context_vector, vector)

   # sorted_x = sorted(vals.items(), key=operator.itemgetter(1), reverse=True)
   # for k,v in sorted_x:
    #    print("%.3f\t%s\t%s"% (v,k,k.definition()))
    
    return(synsets[1].definition())


Execute the following two cells to check whether your implementation distinguishes between these two senses of "bank".

In [39]:
bert_lesk("bank", "I deposited my money into my savings account at the bank", "n")

# This prints! It breaks once I try to index through synsets which is confusing to me. Christian and I were working
# on this and couldn't figure out why that didn't work. 



'a financial institution that accepts deposits and channels the money into lending activities'

In [42]:
from nltk.corpus.reader.wordnet import Synset

def bert_lesk(word, sentence, part_of_speech):

# this function gets the BERT token representations for a given term within a larger string
    def get_bert_for_token(string, term):
        # tokenize
        inputs = tokenizer(string, return_tensors="pt")
        tokens=tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        # find the first location of the query term among those tokens (so we know which BERT rep to use)
        term_idx=tokens.index(term)
        outputs = model(**inputs)
        return outputs.last_hidden_state[0][term_idx].detach().numpy()
    context_vector=get_bert_for_token(sentence, word)
    synsets= wn.synsets(word,pos = part_of_speech)
    
    
    
# lastly, for every one of the synsets in the list, grab the BERT representation between the definition as the string and the input word
    # vals={}
   #  for synset in synsets:
      # code seems to be breaking here; where i get error related to term_idx; this should be vec2?
    #    vector=get_bert_for_token(synset.definition(), word)
    #    vals[synset]=cosine_similarity(context_vector, vector)

   # sorted_x = sorted(vals.items(), key=operator.itemgetter(1), reverse=True)
   # for k,v in sorted_x:
    #    print("%.3f\t%s\t%s"% (v,k,k.definition()))
    
    return(get_bert_for_token(synsets[1].definition(), word))


In [43]:
bert_lesk("bank", "I ran along the river bank", "n")
# getting error that "bank" is not in list but I'm not sure why

ValueError: 'bank' is not in list

Q2.  Now do the same thing with SentenceBERT.  For a gloss $g$ and a target sentence $c$ containing the word to disambiguate, calculate the similarity between them as the cosine similarity of the SentenceBERT vectors of each one:

$$
\cos \left(\textrm{SBERT}(g), \textrm{SBERT}(c) \right)
$$


In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

In [ ]:
def sentencebert_lesk(word, sentence, part_of_speech):

    def get_sentence_bert(string, term):
      embedding=sentence_model.encode(string)
      return embedding

    context_vector = get_sentence_bert(sentence, word)
    synsets= wn.synsets(word,pos = part_of_speech)

    vals={}

    for synset in synsets:
      # code seems to be breaking here; where i get error related to term_idx; this should be vec2?
        vector=get_sentence_bert(synset.definition(), word)
        vals[synset]=cosine_similarity(context_vector, vector)

    sorted_x = sorted(vals.items(), key=operator.itemgetter(1), reverse=True)
    for k,v in sorted_x:
        print("%.3f\t%s\t%s"% (v,k,k.definition()))


Execute the following two cells to check whether your implementation of SentenceBERT-Lesk distinguishes between these two senses of "bank".

In [ ]:
sentencebert_lesk("bank", "I deposited my money into my savings account at the bank", "n")

In [ ]:
sentencebert_lesk("bank", "I ran along the river bank", "n")

To turn in:

- Go to `File > Download > Download .ipynb` and save your notebook.
- In your browser, print this page to save as PDF.
- Upload both your .ipynb and .pdf files to bCourses as usual.